## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pangnirtung,CA,66.1451,-65.7125,-15.21,100,100,5.73,overcast clouds
1,1,Pakxan,LA,18.3778,103.6601,68.72,89,88,3.44,overcast clouds
2,2,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,30,5.12,scattered clouds
3,3,New Norfolk,AU,-42.7826,147.0587,65.70,56,99,4.72,overcast clouds
4,4,Kyren,RU,51.6828,102.1408,-14.55,89,88,1.36,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Pakxan,LA,18.3778,103.6601,68.72,89,88,3.44,overcast clouds
2,2,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,30,5.12,scattered clouds
3,3,New Norfolk,AU,-42.7826,147.0587,65.70,56,99,4.72,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.32,69,1,18.16,light rain
7,7,Cidreira,BR,-30.1811,-50.2056,75.29,79,13,20.38,few clouds
9,9,Puerto Quijarro,BO,-17.7833,-57.7667,85.21,67,75,2.24,broken clouds
10,10,Saint-Joseph,RE,-21.3667,55.6167,77.36,79,18,9.62,few clouds
11,11,Pasighat,IN,28.0667,95.3333,52.00,73,0,4.03,clear sky
15,15,Mataura,NZ,-46.1927,168.8643,50.49,84,98,4.65,overcast clouds
16,16,Cape Town,ZA,-33.9258,18.4232,70.09,79,20,16.11,few clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                412
City                   412
Country                408
Lat                    412
Lng                    412
Max Temp               412
Humidity               412
Cloudiness             412
Wind Speed             412
Current Description    412
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df["Country"].replace('',np.nan, inplace=True)
preferred_cities_df.dropna(subset=["Country"], inplace=True)
preferred_cities_df.count()

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                408
City                   408
Country                408
Lat                    408
Lng                    408
Max Temp               408
Humidity               408
Cloudiness             408
Wind Speed             408
Current Description    408
dtype: int64

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Pakxan,LA,68.72,overcast clouds,18.3778,103.6601,
2,Ponta Do Sol,PT,64.06,scattered clouds,32.6667,-17.1000,
3,New Norfolk,AU,65.70,overcast clouds,-42.7826,147.0587,
6,Rikitea,PF,79.32,light rain,-23.1203,-134.9692,
7,Cidreira,BR,75.29,few clouds,-30.1811,-50.2056,
9,Puerto Quijarro,BO,85.21,broken clouds,-17.7833,-57.7667,
10,Saint-Joseph,RE,77.36,few clouds,-21.3667,55.6167,
11,Pasighat,IN,52.00,clear sky,28.0667,95.3333,
15,Mataura,NZ,50.49,overcast clouds,-46.1927,168.8643,
16,Cape Town,ZA,70.09,few clouds,-33.9258,18.4232,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    #adding params
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Pakxan,LA,68.72,overcast clouds,18.3778,103.6601,Kong Kham Pheng Guesthouse
2,Ponta Do Sol,PT,64.06,scattered clouds,32.6667,-17.1000,Hotel do Campo
3,New Norfolk,AU,65.70,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
6,Rikitea,PF,79.32,light rain,-23.1203,-134.9692,People ThankYou
7,Cidreira,BR,75.29,few clouds,-30.1811,-50.2056,Hotel Castelo


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.count()

City                   385
Country                385
Max Temp               385
Current Description    385
Lat                    385
Lng                    385
Hotel Name             385
dtype: int64

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [44]:
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Kong Kham Pheng Guesthouse</dd>\n<dt>City</dt><dd>Pakxan</dd>\n<dt>Country</dt><dd>LA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 68.72 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel do Campo</dd>\n<dt>City</dt><dd>Ponta Do Sol</dd>\n<dt>Country</dt><dd>PT</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 64.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Shingles Riverside Cottages</dd>\n<dt>City</dt><dd>New Norfolk</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 65.7 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>light rain and 79.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Castelo</dd>\n<dt>City</dt><dd>Cidreira</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>few clouds and 75.29 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hote

In [45]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))